In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-15 13:46:41,565 - NumExpr defaulting to 8 threads.


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'SCS_Alerts_NOV_new.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=3077198555888) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=3077198556320) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=3077198556464) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=3077198553584) was called 1 times
Running: Function "load_module_lookup" (id=3077198554736) was called 1 times
duplicated location faults removed - max duration kept


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [9]:
fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Grey','Both']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'Asset Code', agg_type = 'count')

Running: Function "faults_aggregate" (id=1615276264896) was called 1 times


In [10]:
av_sel,at_sel = feat.av_at_select(av, at, availability_select_options = None, remove_high_AT = False, AT_limit = 50)

av_agg = feat.aggregate_availability(av_sel, agg_level = None)
at_agg = feat.aggregate_totes(at_sel, agg_level = None)

#at_agg = at_agg[at_agg['TOTES']>100]

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  at['TOTES'][i]=AT_limit


Running: Function "aggregate_availability" (id=1615276429376) was called 1 times


In [11]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = None)

In [12]:
X,y = md.gen_feat_var(df,target = 'Grey Tote Loss')
y=1-y

In [13]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    'criterion': 'mse', 
                                                                                    'max_depth': None, 
                                                                                    'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         'max_depth': None})

selection_options= {'thres': -np.inf,
                    'max_feat': None}

X_sel_rf = md.select_features(X, y, RF_mdl, selection_options)

X_sel_lm = md.select_features(X, y, Linear_mdl, selection_options)

X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options)

                                                                                                           #X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,test_size=0.3,random_state=101)

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X_sel_lm, y)

cv_R2 = md.cross_validate_r2(RF_mdl,X_sel_rf, y)

cv_R2 = md.cross_validate_r2(XGB_mdl, X_sel_XGB, y)

#md.stats_model(Linear_mdl,X_test,y_test)

Feature Importance Ranking: 
 
    Importance      Feature
0    0.321013        TOTES
1    0.208531        index
2    0.048925  C1104STA197
3    0.015364       SCSM12
4    0.014938  C0503RDC101
5    0.011904  C1102STA071
6    0.011398       SCSM18
7    0.011383       SCSM03
8    0.011248       SCSM19
9    0.011162  C1404STA164

Regression Metrics: 
 
           RF Metrics
MAE         0.083706
MSE         0.014114
RMSE        0.118802
MAPE%      29.780000
ACC%       70.220000
OOB        -0.042753
R2_Train    0.859138
R2_Pred    -0.002263 

Feature Importance Ranking: 
 
    Importance      Feature
0    0.069414  C1404STA191
1    0.067309  C1202STA064
2    0.065542  C1102STA089
3    0.054523  C1203ACH113
4    0.054022  C1104STA197
5    0.052491  C1102STA064
6    0.052032  C0502STA081
7    0.027015  C1304STA164
8    0.024002  C1202ACH055
9    0.020874  C0503RDC101

Regression Metrics: 
 
          XGB Metrics
MAE        0.0878507
MSE        0.0156998
RMSE        0.125299
MAPE%          28

### Add LM - metrics

In [14]:
sc.lm_coefficients(Linear_mdl, X, y)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,Feature,Coefficients,Standard Errors,t values,Probabilities
0,Constant,27.8702,NaN,NaN,NaN
1,C05,0.0338,NaN,NaN,NaN
2,C0501STA002,0.8079,NaN,NaN,NaN
3,C0501STA012,-7.5595,NaN,NaN,NaN
4,C0501STA217,0.0000,NaN,NaN,NaN
...,...,...,...,...,...
370,SCSM18,0.0122,NaN,NaN,NaN
628,SCSM18,-0.0005,NaN,NaN,NaN
372,SCSM20,0.6558,NaN,NaN,NaN
373,index,-0.0005,NaN,NaN,NaN


In [15]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1698: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1698: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1620: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1446: RuntimeWarning: invalid value encountered in multiply
  cov_p = self.normalized_cov

                            OLS Regression Results                            
Dep. Variable:         Grey Tote Loss   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     3.659
Date:                Tue, 15 Dec 2020   Prob (F-statistic):           2.36e-31
Time:                        11:01:21   Log-Likelihood:                 1483.6
No. Observations:                1498   AIC:                            -2725.
Df Residuals:                    1377   BIC:                            -2082.
Df Model:                         120                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.9192      4.609      1.935      

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [16]:
df_greyhigh_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_ID.sum().sort_values(ascending=False).head(20)

index          2.165556e+08
SCSM18         2.621300e+04
C09            2.498500e+04
C12            2.480300e+04
SCSM12         2.408100e+04
C14            2.098300e+04
C08            2.048600e+04
C13            1.645000e+04
SCSM03         1.545500e+04
SCSM15         1.287400e+04
C11            1.206500e+04
C07            1.204800e+04
C06            1.120500e+04
SCSM04         1.105600e+04
SCSM07         1.091000e+04
SCSM11         1.077400e+04
SCSM01         1.067000e+04
SCSM20         9.621000e+03
C1404STA164    8.889000e+03
SCSM13         8.492000e+03
dtype: float64

In [17]:
df_grey_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_ID.sum().sort_values(ascending=False).head(20)

index          2.165556e+08
SCSM18         2.621300e+04
C09            2.498500e+04
C12            2.480300e+04
SCSM12         2.408100e+04
C14            2.098300e+04
C08            2.048600e+04
C13            1.645000e+04
SCSM03         1.545500e+04
SCSM15         1.287400e+04
C11            1.206500e+04
C07            1.204800e+04
C06            1.120500e+04
SCSM04         1.105600e+04
SCSM07         1.091000e+04
SCSM11         1.077400e+04
SCSM01         1.067000e+04
SCSM20         9.621000e+03
C1404STA164    8.889000e+03
SCSM13         8.492000e+03
dtype: float64

In [18]:
df_greyhigh_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_Asset.sum().sort_values(ascending=False).head(20)

index          2.165556e+08
SCSM18         2.621300e+04
C09            2.498500e+04
C12            2.480300e+04
SCSM12         2.408100e+04
C14            2.098300e+04
C08            2.048600e+04
C13            1.645000e+04
SCSM03         1.545500e+04
SCSM15         1.287400e+04
C11            1.206500e+04
C07            1.204800e+04
C06            1.120500e+04
SCSM04         1.105600e+04
SCSM07         1.091000e+04
SCSM11         1.077400e+04
SCSM01         1.067000e+04
SCSM20         9.621000e+03
C1404STA164    8.889000e+03
SCSM13         8.492000e+03
dtype: float64

In [19]:
df_grey_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_Asset.sum().sort_values(ascending=False).head(20)

index          2.165556e+08
SCSM18         2.621300e+04
C09            2.498500e+04
C12            2.480300e+04
SCSM12         2.408100e+04
C14            2.098300e+04
C08            2.048600e+04
C13            1.645000e+04
SCSM03         1.545500e+04
SCSM15         1.287400e+04
C11            1.206500e+04
C07            1.204800e+04
C06            1.120500e+04
SCSM04         1.105600e+04
SCSM07         1.091000e+04
SCSM11         1.077400e+04
SCSM01         1.067000e+04
SCSM20         9.621000e+03
C1404STA164    8.889000e+03
SCSM13         8.492000e+03
dtype: float64

In [20]:
X

,C05,C0501ACH011,C0501ACH018,C0501STA002,C0501STA012,C0501STA015,C0501STA020,C0501STA217,C0501STA219,C0502ACH079,...,SCSM11,SCSM12,SCSM13,SCSM14,SCSM15,SCSM17,SCSM18,SCSM19,SCSM20,TOTES
27,2,0,0,0,0,0,0,0,0,0,...,4,4,4,0,5,0,4,1,0,60.557407
28,1,0,0,0,0,0,0,0,0,0,...,1,13,1,0,5,0,10,0,0,63.171296
29,4,0,0,0,0,0,0,0,0,0,...,1,21,2,0,0,0,24,0,0,60.872222
30,8,0,0,0,0,0,0,0,0,0,...,15,2,6,4,0,1,2,0,1,66.359259
31,3,0,0,0,0,0,0,0,0,0,...,7,14,5,0,4,0,3,1,6,66.781481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,3,0,0,0,0,0,0,0,0,0,...,1,2,0,0,11,5,7,2,1,32.274328
142,1,0,0,0,0,0,0,0,0,0,...,5,4,0,1,7,0,18,0,3,40.757407
143,6,0,0,0,0,0,0,0,0,0,...,9,15,1,0,5,0,28,5,6,47.532407
144,3,0,0,0,0,0,0,0,0,0,...,4,15,1,2,8,0,18,1,17,45.150000
